# SSA H6 Benchmark Comparison Analysis

This notebook compares PolicyEngine Option 1 (Full Repeal) estimates with Social Security Administration (SSA) H6 benchmarks:

**H6**: Phase out and eliminate federal income taxation of OASDI benefits that is credited to the OASI and DI Trust Funds from 2045-2054

**PolicyEngine Option 1 (Full Repeal)** is a reasonable proxy for H6 since both policies eliminate taxation of Social Security benefits, though H6 phases this in over 2045-2054.

## Analysis Coverage

Decadal analysis (every 10 years) from 2030-2100 showing:
- Total Social Security benefits paid
- Taxable payroll (PolicyEngine vs SSA projections)
- Revenue changes (Option 1 vs H6)

In [1]:
# Import libraries
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")
print(f"✓ Project root: {project_root}")

c:\Users\dtsax\envs\pe\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Libraries imported successfully
✓ Project root: c:\Users\dtsax\PolicyEngine\crfb-tob-impacts


## 1. Define H6 Reform

Create a reform that eliminates Social Security benefit taxation (same as PolicyEngine Option 1 - Full Repeal).

In [2]:
def create_h6_reform():
    """
    Create H6 reform: Eliminate federal income taxation of Social Security benefits.
    
    H6 phases this out from 2045-2054, but for comparison purposes we use
    immediate full repeal (same as PolicyEngine Option 1) starting in 2026.
    """
    reform_dict = {
        # Base rate parameters (0-50% bracket)
        "gov.irs.social_security.taxability.rate.base.benefit_cap": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.base.excess": {
            "2026-01-01.2100-12-31": 0
        },
        # Additional rate parameters (50-85% bracket)
        "gov.irs.social_security.taxability.rate.additional.benefit_cap": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional.bracket": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional.excess": {
            "2026-01-01.2100-12-31": 0
        }
    }
    
    return Reform.from_dict(reform_dict, country_id="us")

h6_reform = create_h6_reform()
print("✓ H6 reform created successfully")
print("\nReform details:")
print("  Eliminates all federal income taxation of Social Security benefits")
print("  Effective date: 2026-01-01")
print("  (H6 phases out 2045-2054; this uses immediate repeal for comparison)")

✓ H6 reform created successfully

Reform details:
  Eliminates all federal income taxation of Social Security benefits
  Effective date: 2026-01-01
  (H6 phases out 2045-2054; this uses immediate repeal for comparison)


## 2. Calculate Revenue Impacts

Calculate the revenue impact of H6 reform (full repeal) for key years to compare against SSA benchmarks.

In [3]:
# Years to analyze (focus on SSA benchmark years)
analysis_years = [2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100]

print("="*80)
print("CALCULATING H6 REVENUE IMPACTS")
print("="*80)

h6_results = []

for year in analysis_years:
    print(f"\nProcessing {year}...")
    
    # Load dataset
    dataset_path = f'hf://policyengine/test/{year}.h5'
    print(f"  Loading dataset: {dataset_path}")
    
    # Baseline simulation
    print(f"  Running baseline simulation...")
    baseline = Microsimulation(dataset=dataset_path)
    baseline_revenue = baseline.calculate("income_tax").sum()
    
    # Reform simulation
    print(f"  Running H6 reform simulation...")
    reform_sim = Microsimulation(dataset=dataset_path, reform=h6_reform)
    reform_revenue = reform_sim.calculate("income_tax").sum()
    
    # Calculate impact
    revenue_impact = reform_revenue - baseline_revenue
    
    # Calculate taxable payroll
    print(f"  Computing taxable payroll...")
    taxable_ss_earnings = baseline.calculate("taxable_earnings_for_social_security")
    taxable_self_employment = baseline.calculate("social_security_taxable_self_employment_income")
    total_taxable_payroll = taxable_ss_earnings.sum() + taxable_self_employment.sum()
    
    # Calculate SS benefits
    print(f"  Computing SS benefits...")
    ss_benefits = baseline.calculate("social_security")
    total_ss_benefits = ss_benefits.sum()
    
    h6_results.append({
        'year': year,
        'baseline_revenue_billions': baseline_revenue / 1e9,
        'reform_revenue_billions': reform_revenue / 1e9,
        'revenue_impact_billions': revenue_impact / 1e9,
        'taxable_payroll_billions': total_taxable_payroll / 1e9,
        'ss_benefits_billions': total_ss_benefits / 1e9,
    })
    
    print(f"  ✓ Baseline Revenue: ${baseline_revenue / 1e9:,.2f}B")
    print(f"  ✓ Reform Revenue: ${reform_revenue / 1e9:,.2f}B")
    print(f"  ✓ Revenue Impact: ${revenue_impact / 1e9:,.2f}B")
    print(f"  ✓ Taxable Payroll: ${total_taxable_payroll / 1e9:,.2f}B")
    print(f"  ✓ SS Benefits: ${total_ss_benefits / 1e9:,.2f}B")

h6_df = pd.DataFrame(h6_results)

print("\n" + "="*80)
print("✓ H6 revenue impact calculation complete")
print("="*80)

CALCULATING H6 REVENUE IMPACTS

Processing 2030...
  Loading dataset: hf://policyengine/test/2030.h5
  Running baseline simulation...
  Running H6 reform simulation...
  Computing taxable payroll...
  Computing SS benefits...
  ✓ Baseline Revenue: $3,434.59B
  ✓ Reform Revenue: $3,290.88B
  ✓ Revenue Impact: $-143.71B
  ✓ Taxable Payroll: $13,239.00B
  ✓ SS Benefits: $2,091.53B

Processing 2040...
  Loading dataset: hf://policyengine/test/2040.h5
  Running baseline simulation...
  Running H6 reform simulation...
  Computing taxable payroll...
  Computing SS benefits...
  ✓ Baseline Revenue: $5,919.77B
  ✓ Reform Revenue: $5,649.66B
  ✓ Revenue Impact: $-270.12B
  ✓ Taxable Payroll: $18,748.35B
  ✓ SS Benefits: $3,242.92B

Processing 2050...
  Loading dataset: hf://policyengine/test/2050.h5
  Running baseline simulation...
  Running H6 reform simulation...
  Computing taxable payroll...
  Computing SS benefits...
  ✓ Baseline Revenue: $9,675.47B
  ✓ Reform Revenue: $9,206.42B
  ✓ Revenu

## 3. Load SSA Benchmark Data

Load SSA's H6 estimates and projected taxable payroll for comparison.

In [4]:
# SSA projected taxable payroll from 2024 Trustees Report Table VI.G6
ssa_projected_payroll = {
    2030: 17_803,
    2040: 27_337,
    2050: 34_419,
    2060: 41_836,
    2070: 50_887,
    2080: 79_177,
    2090: 126_652,
    2100: 191_588,
}

# SSA H6 estimates (change in annual balance as % of taxable payroll)
# Source: https://www.ssa.gov/oact/solvency/provisions/tables/table_run133.html
ssa_h6_pct_payroll = {
    2030: None,  # TODO: Add from SSA table
    2040: None,  # TODO: Add from SSA table
    2050: None,  # TODO: Add from SSA table
    2060: -1.04,
    2070: None,  # TODO: Add from SSA table
    2080: None,  # TODO: Add from SSA table
    2090: None,  # TODO: Add from SSA table
    2100: -1.12,
}

# Add SSA data to results
h6_df['ssa_payroll_billions'] = h6_df['year'].map(ssa_projected_payroll)
h6_df['ssa_h6_pct_payroll'] = h6_df['year'].map(ssa_h6_pct_payroll)
h6_df['ssa_h6_revenue_billions'] = (
    h6_df['ssa_h6_pct_payroll'] / 100 * h6_df['ssa_payroll_billions']
)

# Calculate PE impact as % of SSA taxable payroll
h6_df['pe_pct_ssa_payroll'] = (
    h6_df['revenue_impact_billions'] / h6_df['ssa_payroll_billions'] * 100
)

# Calculate difference
h6_df['diff_pct_payroll'] = (
    h6_df['pe_pct_ssa_payroll'] - h6_df['ssa_h6_pct_payroll']
)
h6_df['diff_revenue_billions'] = (
    h6_df['revenue_impact_billions'] - h6_df['ssa_h6_revenue_billions']
)

print("✓ SSA benchmark data loaded")
print(f"\nSSA projected payroll range: ${ssa_projected_payroll[2030]:,}B (2030) to ${ssa_projected_payroll[2100]:,}B (2100)")

✓ SSA benchmark data loaded

SSA projected payroll range: $17,803B (2030) to $191,588B (2100)


## 4. Comparison Table

Compare PolicyEngine H6 (full repeal) with SSA H6 benchmark.

In [5]:
print("\n" + "="*80)
print("COMPARISON: PolicyEngine H6 (Full Repeal) vs SSA H6 Benchmark")
print("="*80)
print("\nAll revenue figures in billions ($B)")
print("Percentages shown as % of SSA projected taxable payroll\n")

display_df = h6_df[[
    'year',
    'ss_benefits_billions',
    'taxable_payroll_billions',
    'ssa_payroll_billions',
    'revenue_impact_billions',
    'ssa_h6_revenue_billions',
    'diff_revenue_billions',
    'pe_pct_ssa_payroll',
    'ssa_h6_pct_payroll',
    'diff_pct_payroll'
]].copy()

# Format column names
display_df.columns = [
    'Year',
    'SS Benefits ($B)',
    'PE Taxable Payroll ($B)',
    'SSA Taxable Payroll ($B)',
    'PE H6 Impact ($B)',
    'SSA H6 Impact ($B)',
    'Difference ($B)',
    'PE (% SSA Payroll)',
    'SSA (% Payroll)',
    'Diff (pp)'
]

print(display_df.to_string(index=False))


COMPARISON: PolicyEngine H6 (Full Repeal) vs SSA H6 Benchmark

All revenue figures in billions ($B)
Percentages shown as % of SSA projected taxable payroll

 Year  SS Benefits ($B)  PE Taxable Payroll ($B)  SSA Taxable Payroll ($B)  PE H6 Impact ($B)  SSA H6 Impact ($B)  Difference ($B)  PE (% SSA Payroll)  SSA (% Payroll)  Diff (pp)
 2030       2091.526400             13238.999979                     17803        -143.710520                 NaN              NaN           -0.807226              NaN        NaN
 2040       3242.922007             18748.351792                     27337        -270.117639                 NaN              NaN           -0.988103              NaN        NaN
 2050       4802.506797             24463.180125                     34419        -469.047401                 NaN              NaN           -1.362757              NaN        NaN
 2060       7309.190397             30645.201582                     41836        -800.021661           -435.0944      -364.92

## 5. Visualization

Visualize the comparison between PolicyEngine H6 (full repeal) and SSA H6 benchmark.

In [6]:
# Create comparison visualizations
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Revenue Impact: PE H6 vs SSA H6',
        'Revenue Impact as % of SSA Payroll',
        'Total Social Security Benefits',
        'Taxable Payroll: PE vs SSA Projections'
    ),
    vertical_spacing=0.15,
    horizontal_spacing=0.12
)

years = h6_df['year']
ssa_data = h6_df[h6_df['ssa_h6_revenue_billions'].notna()]

# 1. Revenue Impact (Dollar amounts)
fig.add_trace(
    go.Scatter(
        x=years,
        y=h6_df['revenue_impact_billions'],
        mode='lines+markers',
        name='PolicyEngine H6',
        line=dict(color='#2E86AB', width=3),
        marker=dict(size=8),
        text=h6_df['revenue_impact_billions'].apply(lambda x: f"${x:.1f}B"),
        hovertemplate='<b>Year:</b> %{x}<br><b>PE H6:</b> %{text}<extra></extra>'
    ),
    row=1, col=1
)

if len(ssa_data) > 0:
    fig.add_trace(
        go.Scatter(
            x=ssa_data['year'],
            y=ssa_data['ssa_h6_revenue_billions'],
            mode='markers',
            name='SSA H6 Benchmark',
            marker=dict(color='#A23B72', size=12, symbol='diamond'),
            text=ssa_data['ssa_h6_revenue_billions'].apply(lambda x: f"${x:.1f}B"),
            hovertemplate='<b>Year:</b> %{x}<br><b>SSA H6:</b> %{text}<extra></extra>'
        ),
        row=1, col=1
    )

# 2. Revenue Impact (% of payroll)
fig.add_trace(
    go.Scatter(
        x=years,
        y=h6_df['pe_pct_ssa_payroll'],
        mode='lines+markers',
        name='PolicyEngine H6',
        line=dict(color='#2E86AB', width=3),
        marker=dict(size=8),
        showlegend=False,
        text=h6_df['pe_pct_ssa_payroll'].apply(lambda x: f"{x:.3f}%"),
        hovertemplate='<b>Year:</b> %{x}<br><b>PE:</b> %{text}<extra></extra>'
    ),
    row=1, col=2
)

if len(ssa_data) > 0:
    fig.add_trace(
        go.Scatter(
            x=ssa_data['year'],
            y=ssa_data['ssa_h6_pct_payroll'],
            mode='markers',
            name='SSA H6',
            marker=dict(color='#A23B72', size=12, symbol='diamond'),
            showlegend=False,
            text=ssa_data['ssa_h6_pct_payroll'].apply(lambda x: f"{x:.3f}%"),
            hovertemplate='<b>Year:</b> %{x}<br><b>SSA:</b> %{text}<extra></extra>'
        ),
        row=1, col=2
    )

# 3. Total SS Benefits
fig.add_trace(
    go.Scatter(
        x=years,
        y=h6_df['ss_benefits_billions'],
        mode='lines+markers',
        name='SS Benefits',
        line=dict(color='#06BEE1', width=3),
        marker=dict(size=8),
        showlegend=False,
        text=h6_df['ss_benefits_billions'].apply(lambda x: f"${x:,.0f}B"),
        hovertemplate='<b>Year:</b> %{x}<br><b>Benefits:</b> %{text}<extra></extra>'
    ),
    row=2, col=1
)

# 4. Taxable Payroll comparison
fig.add_trace(
    go.Scatter(
        x=years,
        y=h6_df['taxable_payroll_billions'],
        mode='lines+markers',
        name='PolicyEngine',
        line=dict(color='#2E86AB', width=3),
        marker=dict(size=8),
        showlegend=False
    ),
    row=2, col=2
)

fig.add_trace(
    go.Scatter(
        x=years,
        y=h6_df['ssa_payroll_billions'],
        mode='lines+markers',
        name='SSA Projection',
        line=dict(color='#F18F01', width=3, dash='dash'),
        marker=dict(size=8),
        showlegend=True
    ),
    row=2, col=2
)

# Update axes
fig.update_yaxes(title_text="Revenue Impact ($B)", row=1, col=1)
fig.update_yaxes(title_text="% of SSA Payroll", row=1, col=2)
fig.update_yaxes(title_text="Billions ($)", row=2, col=1)
fig.update_yaxes(title_text="Taxable Payroll ($B)", row=2, col=2)

for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(title_text="Year", row=i, col=j)

# Update layout
fig.update_layout(
    height=900,
    title_text="H6 Analysis: PolicyEngine (Full Repeal) vs SSA H6 Benchmark<br><sub>Eliminate federal income taxation of SS benefits</sub>",
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()

## 6. Export Results

In [7]:
# Create output directory if it doesn't exist
output_dir = project_root / 'data'
output_dir.mkdir(exist_ok=True)

# Export H6 comparison
output_file = output_dir / 'h6_comparison.csv'
h6_df.to_csv(output_file, index=False)
print(f"✓ H6 comparison exported to: {output_file}")

# Display summary
print("\n" + "="*80)
print("KEY FINDINGS")
print("="*80)

ssa_years = h6_df[h6_df['ssa_h6_revenue_billions'].notna()]
for _, row in ssa_years.iterrows():
    print(f"\nYear {row['year']}:")
    print(f"  PolicyEngine H6: ${row['revenue_impact_billions']:.2f}B ({row['pe_pct_ssa_payroll']:.3f}% of SSA payroll)")
    print(f"  SSA H6 Benchmark: ${row['ssa_h6_revenue_billions']:.2f}B ({row['ssa_h6_pct_payroll']:.3f}% of SSA payroll)")
    print(f"  Difference: ${row['diff_revenue_billions']:+.2f}B ({row['diff_pct_payroll']:+.3f} percentage points)")
    print(f"  SSA projects: ${row['ssa_payroll_billions']:,.0f}B taxable payroll")
    print(f"  PolicyEngine: ${row['taxable_payroll_billions']:,.2f}B taxable payroll")

print("\n✓ H6 analysis complete!")

✓ H6 comparison exported to: c:\Users\dtsax\PolicyEngine\crfb-tob-impacts\data\h6_comparison.csv

KEY FINDINGS

Year 2060.0:
  PolicyEngine H6: $-800.02B (-1.912% of SSA payroll)
  SSA H6 Benchmark: $-435.09B (-1.040% of SSA payroll)
  Difference: $-364.93B (-0.872 percentage points)
  SSA projects: $41,836B taxable payroll
  PolicyEngine: $30,645.20B taxable payroll

Year 2100.0:
  PolicyEngine H6: $-5215.22B (-2.722% of SSA payroll)
  SSA H6 Benchmark: $-2145.79B (-1.120% of SSA payroll)
  Difference: $-3069.44B (-1.602 percentage points)
  SSA projects: $191,588B taxable payroll
  PolicyEngine: $47,854.61B taxable payroll

✓ H6 analysis complete!
